In [ ]:
import sys
sys.version

In [ ]:
try:
    from google.colab import output
    !pip install rai==0.2.0-dev5
    
except ModuleNotFoundError:
    %load_ext autoreload
    %autoreload 2

In [ ]:
import pathlib
from io import BytesIO
import zipfile
import tqdm
import numpy as np
import skimage.measure
import matplotlib.pyplot as plt

In [ ]:
import pydicom
pydicom.__version__

In [ ]:
import rai
rai.__version__

In [ ]:
import raicontours

from raicontours import TG263

raicontours.__version__

In [ ]:
cfg = raicontours.get_config()

In [ ]:
cfg

In [ ]:
# from rai.inference._weighting import create_inference_weighting

In [ ]:
# weighting = create_inference_weighting((64,64,64))
# plt.figure()
# plt.pcolormesh(weighting[32,...,0], vmin=0, vmax=1)
# plt.axis('equal')
# print(np.min(weighting), np.mean(weighting), np.max(weighting))

# plt.figure()
# plt.pcolormesh(weighting[16,...,0], vmin=0, vmax=1)
# plt.axis('equal')

In [ ]:
# weighting.shape

In [ ]:
# TODO: Save training, validation, and hold out uids within raicontours
# Have this downloader be able to receive "training", "validation", or "hold out"
# Have a random one be used per user.
image_paths, structure_path, data_license_path, data_readme_path, rai_license_path = rai.download_hnscc_example()
structure_path

In [ ]:
sorted_image_paths = rai.sort_dicom_image_paths(paths=image_paths)

# Prune the images used just for this demo (makes final download size smaller)
# selected_sorted_image_paths = sorted_image_paths[14:78]

# Use the following to instead use the entire image stack:
selected_sorted_image_paths = sorted_image_paths

In [ ]:
sorted_image_series = [pydicom.dcmread(path) for path in selected_sorted_image_paths]

In [ ]:
grids, image_stack = rai.sorted_image_series_to_image_stack_hfs(
    cfg=cfg, sorted_image_series=sorted_image_series)

In [ ]:
models = rai.load_models(cfg=cfg)

In [ ]:
# TODO: Add in a refinement step based on points where there is uncertainty (ie artefact regions)
# * Collect min and max values for each pixel. Points with enough difference get further refined.
# * Consider adding in metal streak artefacts during training.

predicted_masks = rai.inference(
    cfg=cfg,
    models=models,
    image_stack=image_stack,
    max_batch_size=10,
)

In [ ]:
# np.vstack(diff).T.tolist()

In [ ]:
0.975 * 255

In [ ]:
255 / 2

In [ ]:
vmin = 0.22
vmax = 0.35
figsize = (8, 8)

# slice_indices, axis_limits, centre_indices = rai.view_ranges_from_masks(grids, predicted_masks)



In [ ]:
# def callback(fig, ax, i):
#     if i == 47:
#         print(i)
#         ax.tick_params(
#             axis='both',     
#             which='both',  
#             bottom=False,  
#             top=False,     
#             left=False,
#             right=False,
#             labelbottom=False,
#             labelleft=False
#         )
#         ax.get_legend().remove()
#         fig.tight_layout()
#         ax.margins(x=0, y=0)
#         ax.set_position([0, 0, 1, 1])
        
#         plt.savefig('image.png', bbox_inches='tight', pad_inches=0)

In [ ]:
from rai.display.animation import view_ranges_from_contours_by_structure


In [ ]:
cfg = raicontours.get_config()

predicted_contours_by_structure = rai.masks_to_contours_by_structure(
    cfg=cfg,
    grids=grids, 
    masks=predicted_masks, 
    structure_names=cfg["structures"],
)

slice_indices, axis_limits = view_ranges_from_contours_by_structure(predicted_contours_by_structure, buffer=20)

rai.plot_contours_by_structure(
    grids=grids,
    images=image_stack,
    contours_by_structure=predicted_contours_by_structure,
    orientation="transverse",
    slice_indices=slice_indices,
    axis_limits=axis_limits,
    structure_names=cfg["structures"],
    figsize=figsize,
    vmin=vmin,
    vmax=vmax,
    # callback=callback
)

In [ ]:
created_structure_ds = rai.create_dicom_structure_set(
    cfg=cfg, 
    image_series_headers=sorted_image_series, 
    contours_by_structure=predicted_contours_by_structure
)

In [ ]:
created_structure_path = pathlib.Path("RS-RAi.dcm")

pydicom.dcmwrite(filename=created_structure_path, dataset=created_structure_ds, write_like_original=False)

In [ ]:
save_to_images_directory = selected_sorted_image_paths + [data_license_path, data_readme_path]
save_to_structure_directory = [rai_license_path, created_structure_path]

with zipfile.ZipFile('results.zip', 'w', compression=zipfile.ZIP_DEFLATED) as results_zip:
    for path in save_to_images_directory:
        results_zip.write(path, f"HNSCC/{path.name}")
        
    for path in save_to_structure_directory:
        results_zip.write(path, f"RAi/{path.name}")

In [ ]:
# Provide the following link to users to view the DICOM files

# https://viewer.imaging.datacommons.cancer.gov/local

In [ ]:
structure_ds = pydicom.read_file(structure_path)
[item.ROIName for item in structure_ds.StructureSetROISequence]

In [ ]:
image_uids = [ds.SOPInstanceUID for ds in sorted_image_series]

dicom_structures_to_plot = [
        'Lens', 'Eyes', 'R Optic Nerve', 'L Optic Nerve'
    ]

dicom_contours_by_structure = rai.dicom_to_contours_by_structure(
    ds=structure_ds, image_uids=image_uids, structure_names=dicom_structures_to_plot
)

In [ ]:
rai.plot_contours_by_structure(
    grids=grids,
    images=image_stack,
    contours_by_structure=dicom_contours_by_structure,
    orientation="transverse",
    slice_indices=slice_indices,
    axis_limits=axis_limits,
    structure_names=dicom_structures_to_plot,
    figsize=figsize,
    vmin=vmin,
    vmax=vmax,
    callback=callback,
)

In [ ]:
# aligned_predicted_contours_by_structure = rai.merge_contours_by_structure(
#     predicted_contours_by_structure, align_map
# )

# dice = {}
# for name in align_map:
#     dice[name] = rai.dice_from_contours_by_slice(
#         dicom_contours_by_structure[name],
#         aligned_predicted_contours_by_structure[name],
#     )
    
# dice

In [ ]:
# TODO:
# * Calculate and display hausdorff
# * Turn into DICOM file, use online web DICOM viewer to view the file